In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import os,sys
import utils
import matplotlib.pyplot as plt
from consts import sensor_lat as lat
from consts import sensor_log as log
from consts import valid_sensor_res_10 as valid_sensor_list
from numpy import array
import time
from multiprocessing import Pool

In [ ]:
# df = pd.read_hdf('data.h5','df_sub')

In [2]:
unix_column = 'Time since the epoch [s]'
idd = 'Station ID'

In [3]:
def coverage_test(df,time_stamps,valid_sensor_list):
    error = 0
    # all_np_list = []
    for i in range(len(time_stamps)-1):
        # print(i)
        # print(time_stamps[i],time_stamps[i+1])
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] < time_stamps[i+1])]
        sensor_list = sub_df[idd].values
        uniq = np.unique(sensor_list)
        diff = np.setdiff1d(valid_sensor_list,uniq)    
        # np_court = all_np(sensor_list)
        # all_np_list.append(np_court)
        error += len(diff)
    return error

In [4]:
def compute(start_point,valid_sensor_list):
    df = pd.read_hdf('day.h5',str(start_point))
    start_time = time.time()
    time_stamps = np.arange(start_point,start_point+week_second+epoch_length,epoch_length)
    error_num = 0
    # all_np_list = []
    for i in range(len(time_stamps)-1):
        # print(i)
        # print(time_stamps[i],time_stamps[i+1])
        sub_df = df[(df[unix_column] > time_stamps[i]) & (df[unix_column] < time_stamps[i+1])]
        sensor_list = sub_df[idd].values
        uniq = np.unique(sensor_list)
        diff = np.setdiff1d(valid_sensor_list,uniq)    
        # np_court = all_np(sensor_list)
        # all_np_list.append(np_court)
        error_num += len(diff)
    now = time.gmtime(start_point)
    print("{0} \t {1}-{2}-{3}:\t {4} \t{5}".format(start_point,now.tm_year,now.tm_mon,now.tm_mday,error_num,time.time()-start_time))
    return (start_point,now,error_num)

In [5]:
day_second = 86400
week_second = 7*day_second
start_list = np.arange(1162393768,1178747998+day_second,day_second)
epoch_length = 1800
# start_list = start_list[:1]
print(start_list)

[1162393768 1162480168 1162566568 1162652968 1162739368 1162825768
 1162912168 1162998568 1163084968 1163171368 1163257768 1163344168
 1163430568 1163516968 1163603368 1163689768 1163776168 1163862568
 1163948968 1164035368 1164121768 1164208168 1164294568 1164380968
 1164467368 1164553768 1164640168 1164726568 1164812968 1164899368
 1164985768 1165072168 1165158568 1165244968 1165331368 1165417768
 1165504168 1165590568 1165676968 1165763368 1165849768 1165936168
 1166022568 1166108968 1166195368 1166281768 1166368168 1166454568
 1166540968 1166627368 1166713768 1166800168 1166886568 1166972968
 1167059368 1167145768 1167232168 1167318568 1167404968 1167491368
 1167577768 1167664168 1167750568 1167836968 1167923368 1168009768
 1168096168 1168182568 1168268968 1168355368 1168441768 1168528168
 1168614568 1168700968 1168787368 1168873768 1168960168 1169046568
 1169132968 1169219368 1169305768 1169392168 1169478568 1169564968
 1169651368 1169737768 1169824168 1169910568 1169996968 117008

In [6]:
results = []
p = Pool(4)
for start_point in start_list:
    # print('assign\t',str(start_point))
    result = p.apply_async(func=compute,args=(start_point,valid_sensor_list,))
    results.append(result)
p.close()
p.join()

KeyboardInterrupt: 